# Table of Contents
 <p><div class="lev1"><a href="#summary"><span class="toc-item-num">1&nbsp;&nbsp;</span>summary</a></div><div class="lev2"><a href="#Direct-view"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Direct view</a></div><div class="lev2"><a href="#LoadBalancedView"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>LoadBalancedView</a></div><div class="lev3"><a href="#restrict-calculation-time"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>restrict calculation time</a></div>

# summary

In [1]:
from __future__ import print_function, division # legacy support
import pandas as pd
import numpy as np

In [2]:
%matplotlib inline

## Direct view

Direct view is a simple way to perform parallel calculations in ipython, however this has some drawbacks:

Advantages:
- Easy

Disadvantages:
- While performing the parallel calculations, the notebook kernel is busy. This means that you can change whatever you want in the notebook, but you have to wait until the parallel calculation has finished before you can evaluate other cells. (So for example during the parallel calculation you want to make a figure, however the evaluation of the figure code will only be done after finishing the parallel calculation).
- For the direct view you have to 'scatter' your scenarios before you can start the simulations. This can be a drawback in some cases, for example: You have 10 cores and 1000 scenarios you want to calculate. So when you use the scatter command, you will give 100 scenarios to every core. Seems reasonable, right? However, imagine that some scenarios are really fast and others are really slow... this can have as an impact that some cores will be finished, while the others still have to do a lot of work.
- Less options


In [3]:
from IPython.parallel import Client             # Import client for parallel calculation

c = Client(profile='nbserver')                  # Make client using the cores started with --profile='nbserver'
dview = c[:]                                    # Make directView of all clients, used to parse data/objects to cores
print(c.ids)                                    # Print all clients

[0, 1, 2, 3]


/home/jos/anaconda/envs/ipyparallelseminar/lib/python3.6/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [103]:
%%px --local
# Execute on all cores but also for serial calculations, has to be loaded on the FIRST line
# This code should only be runned when using self defined classes 
# Load dill, More general pickling class
import dill
# fallback to pickle instead of cPickle, so that dill can take over
import pickle
from IPython.kernel.zmq import serialize
serialize.pickle = pickle

In [5]:
# Import on all cores
with c[:].sync_imports():
    import math

importing math on engine(s)


There are two options to load your functions and other variables on the different cores
1. Use "%%px --local" to load everything which is defined in the cells both in the local and the core environments
2. Use "dview['data'] = data" to parse local variables to all the different cores

In [6]:
%%px --local
def get_sum(pararray = None):
    return sum(pararray)

In [48]:
samples = 5000
scenarios = np.random.random([samples,3])

In [51]:
# Example for using 4 parallel cores
def mul(a, b):
    return a * b

In [52]:
res = dview.map(mul, [5, 6, 7, 8], [8, 9, 10, 11])

In [53]:
res.result()

[40, 54, 70, 88]

In [54]:
res = dview.apply(mul, 5, 6)

In [55]:
res.get()

[30, 30, 30, 30]

In a DirectView interface, we can either use Blocking (synchronous) execution, in which all results must finish computing 
before any results are recorded, or non-blocking (asynchronous) execution, where we receive results as they finish.

- Get results directly without using the .result option
    <pre><code>
    out = dview.map_sync(get_sum, scenarios)
    </code></pre>
- Create objects for which you have to use the .result option
    <pre><code>
    out = dview.map_async(get_sum, scenarios)
    </code></pre>

However only the number of runned scenarios will equal the number of processors,
therefore we have to "scatter" the scenarios over the different cores

In [56]:
dview.scatter('scenarios',scenarios)

<AsyncResult: scatter>

In [57]:
%px out = [get_sum(scen) for scen in scenarios]

In [58]:
out = dview.gather('out')
out = out.get()

In [59]:
df_out = pd.DataFrame(np.hstack([scenarios, np.atleast_2d(out).T]), 
             columns=['par1','par2','par3','sum'])

In [60]:
df_out

,par1,par2,par3,sum
0,0.699607,0.175533,0.246082,1.121222
1,0.498231,0.256507,0.621850,1.376588
2,0.164486,0.297286,0.059924,0.521696
3,0.832955,0.665147,0.077162,1.575264
4,0.350940,0.238428,0.974222,1.563590
5,0.310637,0.931881,0.993904,2.236421
6,0.493271,0.718810,0.742462,1.954544
7,0.422168,0.460127,0.947626,1.829921
8,0.510682,0.810674,0.493296,1.814653
9,0.598862,0.657733,0.101053,1.357647


## LoadBalancedView

LoadBalancedView is a more advanced way of performing parallel calculations

Advantages:
- Parallel calculations are not directly connected to your notebook, this means that during the parallel calculations your notebook is as productive as usual. For very fast functions the difference will be non-existing (or even the other way around), but when load increases you will see the advantage of the LoadBalancedView.
- The addressed cores are used in the most efficient way, because the assignment of the work to the different cores is done dynamically during the calculation. Thereby avoiding unbalanced workload of the different cores.
- More options
- All the stuff which is printed during each simulation is not shown, keeping your notebook tidy and clean ^^

Disadvantages:
- No printing during calculation, errors are not popping up. So if something goes wrong with simulations, this will not directly show you an error. The advantage is that the next calculation is started automatically, so no loss of time. The disadvantage is that when all simulations are failing, this only pops up when you ask for the results.

In a loadBalancedView interface, we can either use Blocking (synchronous) execution, in which all results must finish computing 
before any results are recorded, or non-blocking (asynchronous) execution, where we receive results as they finish.

- Get results directly without using the .result option
    <pre><code>
    out = lview.map_sync(get_sum, scenarios)
    </code></pre>
- Create objects for which you have to use the .result option
    <pre><code>
    out = lview.map_async(get_sum, scenarios)
    </code></pre>

In [65]:
out = lview.map_async(get_sum, scenarios)

In [99]:
out.progress

5000

In [101]:
df_out = pd.DataFrame(np.hstack([scenarios, np.atleast_2d(out.result()).T]), 
             columns=['par1','par2','par3','sum'])

In [102]:
df_out

,par1,par2,par3,sum
0,0.699607,0.175533,0.246082,1.121222
1,0.498231,0.256507,0.621850,1.376588
2,0.164486,0.297286,0.059924,0.521696
3,0.832955,0.665147,0.077162,1.575264
4,0.350940,0.238428,0.974222,1.563590
5,0.310637,0.931881,0.993904,2.236421
6,0.493271,0.718810,0.742462,1.954544
7,0.422168,0.460127,0.947626,1.829921
8,0.510682,0.810674,0.493296,1.814653
9,0.598862,0.657733,0.101053,1.357647
